## 0. Setup

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import re
from dbfread import DBF

In [2]:
# CT_data: https://borealisdata.ca/dataverse/unicen_aggregate
# Borders_data: https://borealisdata.ca/dataverse/unicen_boundaries

ct_borders_1976 = gpd.read_file("../data_raw/CT_data/1976/ct_1976.geojson")
ct_census_1976 = pd.DataFrame(DBF("../data_raw/CT_data/1976/census_wide_1976_ct.dbf"))

ct_borders_1981 = gpd.read_file("../data_raw/CT_data/1981/ct_1981.geojson")
ct_census_1981 = pd.read_csv("../data_raw/CT_data/1981/census_wide_1981_ct.csv")

ct_borders_1986 = gpd.read_file("../data_raw/CT_data/1986/ct_1986.geojson")
ct_census_1986 = pd.read_csv("../data_raw/CT_data/1986/census_wide_1986_ct.csv")

ct_borders_1991 = gpd.read_file("../data_raw/CT_data/1991/ct_1991.geojson")
ct_census_1991 = pd.read_csv("../data_raw/CT_data/1991/census_wide_1991_ct.csv")

ct_borders_1996 = gpd.read_file("../data_raw/CT_data/1996/ct_1996.geojson")
ct_census_1996 = pd.read_csv("../data_raw/CT_data/1996/census_wide_1996_ct.csv")

In [3]:
years = {
    "1981": [ct_borders_1981, ct_census_1981], 
    "1986": [ct_borders_1986, ct_census_1986], 
    "1991": [ct_borders_1991, ct_census_1991],
    "1996": [ct_borders_1996, ct_census_1996]
}


## 1.01: How to find the columns we want

In [4]:
# Download and open the codebook, then search the heading we want, get the corresponding code (e.g. Population density: dnk22021ttn)

## 1.1: Collect only the columns that we want

In [5]:

# - Population density: dnk2{year}ttn
# - Dwelling types: dwtp
# - Number of bedrooms: dwbd
# - Tenure: hhtn

In [6]:
def get_columns(pattern, census_columns):
    # Function to get the column names based on a pattern
    matches = census_columns.str.findall(pattern)
    columns_extracted = matches[matches.apply(bool)].index
    return list(columns_extracted)

In [7]:
def extracted_census(census):
    # Gets the columns that we want based on the t_theme code
    
    census_columns = census.columns.to_series()
    
    # dwellings = get_columns(r"^dw") This is for all dwelling information
    density = get_columns(r"^dnk2", census_columns)
    population = get_columns(r"^pop", census_columns)
    dw_types = get_columns(r"^dwtp", census_columns)
    household = get_columns(r"^hhtn", census_columns)
    num_bedrooms = get_columns(r"^dwbd", census_columns)
    
    all_cols = ["geosid"] + density + population + dw_types + household + num_bedrooms
    return census[all_cols]

In [8]:
def joining_data(census, borders):
    # Joins the Census and the Boundary data together on geosid
    borders["geosid"] = borders["geosid"].astype(float)
    return_df = census.merge(borders, on="geosid")
    return return_df

In [9]:
def data_extraction(census, boundaries, rename_dict):
    ct = extracted_census(census)
    ct = ct.rename(columns={k: v for k, v in rename_dict.items() if k in ct.columns})
    ct = ct[["geosid"] + [col for col in rename_dict.values() if col in ct.columns]]
    joined = joining_data(ct, boundaries)
    ct["Population Density per square kilometre"] = joined["Total Population"] / joined["areakm"]
    ct = ct.drop("Total Population", axis = 1)
    return ct

In [10]:
complete_data = []

for i in years.keys():
    year = str(i)

    rename_dict = {
        f'dnk2{year}ttn': 'Population Density per square kilometre',
        f'pop__tot{year}ttd': "Total Population",
        f'dwtp_tot{year}ttd': 'Total Occupied Private Dwellings',
        f'dwtpsdet{year}tt1': "Single-detached house",
        f'dwtpatchsemi{year}tt1': "Semi-detached house",
        f'dwtpatchrow_{year}tt1': "Row house",
        f'dwtpdupl{year}tt1': "Apartment, duplex",
        f'dwtpapt0{year}tt1': 'Apartment, building that has fewer than five storeys',
        f'dwtpapt5{year}tt1': 'Apartment, building that has five or more storeys',
        f'dwtpmove{year}tt1': 'Movable dwelling',
        f'dwtpothr{year}tt1': 'Other dwelling',
        
        f'dwbd0000{year}tt1': "No bedrooms",
        f'dwbd0001{year}tt1': "1 bedroom",
        f'dwbd0002{year}tt1': "2 bedrooms",
        f'dwbd0003{year}tt1': "3 bedrooms",
        f'dwbd0004{year}tt1': "4 bedrooms",
        f'dwbd005p{year}tt1': "5 or more bedrooms",
        
        f"dwbd_avg{year}ttn": "Average number of bedrooms per dwelling",
        # f"hhtn_tot{year}ttd": "Total Private Households",          Is this useful?
        f'hhtnown_{year}tt1': 'Owned',
        f'hhtnrent{year}tt1': 'Rented',
    }

    complete_data.append(data_extraction(years[i][1], years[i][0], rename_dict))

In [11]:
rename_dict_1976 = {
    'pop_003d76': 'Total Population',
    'dwtp003d76': 'Total Occupied Private Dwellings', 
    'dwtp006176': 'Apartment', 
    'dwtp009176': 'Row house', 
    'dwtp012176': 'Semi-detached house',
    'dwtp015176': 'Apartment, duplex',
    'dwtp018176': 'Movable dwelling',
    'dwtp021176': 'Single-detached house',
    #'hhtn003d76': "Total Private Households",          Is this useful?
    'hhtn006176': 'Owned',
    'hhtn009176': 'Rented'
}
ct_census_1976["geosid"] = ct_census_1976["geosid"].astype(float)
ct_borders_1976 = ct_borders_1976.drop(index=[2310, 2484])
ct1976 = data_extraction(ct_census_1976, ct_borders_1976, rename_dict_1976)

## 1991 Edits

In [12]:
complete_data[2]["4 or more bedrooms"] = complete_data[2]["5 or more bedrooms"] + complete_data[2]["4 bedrooms"]
complete_data[2]["0 to 1 bedroom"] = complete_data[2]["No bedrooms"] + complete_data[2]["1 bedroom"]
# complete_data[2] = complete_data[2].drop(["5 or more bedrooms", "4 bedrooms", "No bedrooms", "1 bedroom"], axis=1)

In [13]:
# Adds this information since it doesn't exist in UNICEN for 1991
ct1991_extra = pd.read_csv("../data_raw/CT_data/1991/ct1991.csv", skiprows = 693)

In [14]:
dwelling_types = pd.concat([ct1991_extra.iloc[:,[1,2]],ct1991_extra.iloc[:,87:95]], axis=1)
dwelling_types["geosid"] = dwelling_types["COL1"] * 10000 + dwelling_types["COL2"]
ct1991 = complete_data[2].merge(dwelling_types, on = "geosid")

In [15]:
ct1991 = ct1991.drop(["Single-detached house", 
                      "Apartment, building that has five or more storeys", 
                      "Movable dwelling", "COL1", "COL2"], axis=1)

rename_dict_1991 = {
        "COL87": 'Single-detached house',
        "COL88": 'Semi-detached house',
        "COL89": 'Row house',
        "COL90": 'Apartment, duplex',
        "COL91": 'Apartment, building that has five or more storeys',
        "COL92": 'Apartment, building that has fewer than five storeys',
        "COL93": 'Other single-attached house',
        "COL94": 'Movable dwelling',
    }

ct1991 = ct1991.rename(columns=rename_dict_1991)

## Export as csv

In [16]:
ct1976.to_csv('../data_raw/cleaned_data/1976_census_data.csv', index=False)
complete_data[0].to_csv('../data_raw/cleaned_data/1981_census_data.csv', index=False)
complete_data[1].to_csv('../data_raw/cleaned_data/1986_census_data.csv', index=False)
ct1991.to_csv('../data_raw/cleaned_data/1991_census_data.csv', index=False)
complete_data[3].to_csv('../data_raw/cleaned_data/1996_census_data.csv', index=False)